<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/011_Multi_Agent_Feedback_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🔁 Multi-Agent Feedback Loop

#### 1. **Writer Agent**
- Generates a first draft (e.g., definition of an economic indicator in JSON format).

#### 2. **Critic Agent**
- Reviews the draft.
- Highlights **gaps, clarity issues, verbosity, accuracy problems**, etc.
- Returns a structured critique.

#### 3. **Editor Agent**
- Takes both the original and the critique.
- Produces an improved version based on feedback.

---

# 🔁 Multi-Agent Feedback Loop Framework

## 🎯 Purpose
Build a **self-improving multi-agent system** to define and refine economic indicators, suitable for blog posts, client-facing reports, and intelligent research assistants.

---

## 🔧 Agent Overview

### 1. 📝 Writer Agent *(Already Implemented)*  
Generates a **first-draft structured definition** of an economic indicator.  
Returns a **valid JSON object** with the following keys:
```json
{
  "indicator": "...",
  "definition": "...",
  "importance": "...",
  "signals": "..."
}
```

✅ **Selected Model**: `MBZUAI/LaMini-Flan-T5-783M`  
✅ **Why**: Strong performance, coherent writing, valid JSON structure, Colab-friendly size.  
✅ **Strategy**: Instead of forcing models to write in JSON, we *post-parse* natural language using Python if needed.

---

### 2. 🧠 Critic Agent *(Coming Next)*  
Analyzes the Writer Agent’s JSON output.  
Returns **structured, constructive feedback** with the following fields:
```json
{
  "accuracy": "...",
  "clarity": "...",
  "completeness": "...",
  "suggestions": "..."
}
```






In [ ]:
!pip install -q transformers huggingface_hub

In [ ]:
import json
import logging
from transformers import pipeline

# Suppress warning logs
logging.getLogger("transformers").setLevel(logging.ERROR)

# ✅ Model Selection (single best performer)
MODEL_NAME = "MBZUAI/LaMini-Flan-T5-783M"

# ✅ Prompt Generator
def get_definition_prompt(indicator):
    return f"""
You are a financial assistant helping a researcher understand key economic indicators.

Define the indicator: {indicator}

Write a clear, informative paragraph that explains:
- What it is
- Why it matters
- What it signals about the economy

Write in a blog-style tone suitable for non-experts.
""".strip()

# ✅ Run Writer Agent
def generate_definition(indicator):
    prompt = get_definition_prompt(indicator)
    generator = pipeline("text2text-generation", model=MODEL_NAME)
    output = generator(prompt, max_new_tokens=300)[0]['generated_text'].strip()

    return {
        "indicator": indicator,
        "definition": output
    }

# ✅ Example: Generate definition for a test indicator
# ✅ Example: Generate definition for a test indicator
if __name__ == "__main__":
    indicator_name = "10-Year Treasury Yield"
    result = generate_definition(indicator_name)

    print("\n" + "="*40)
    print("✅ Writer Agent Output (Unified JSON)")
    print("="*40 + "\n")
    print(json.dumps(result, indent=2))


✅ Writer Agent Output (Unified JSON)

{
  "indicator": "10-Year Treasury Yield",
  "definition": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}


> ✅ Writer Agent Output (Unified JSON):

{
  "indicator": "10-Year Treasury Yield",
  "definition": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}

### 🧠 Multi-Agent Workflow for Economic Indicator Blog Posts

| Agent Role            | Task |
|-----------------------|------|
| `DefinitionAgent`     | Defines the indicator: what it is, why it matters, and what it signals about the economy. |
| `ReviewAgent`         | Analyzes the definition for completeness, clarity, and correctness. May add caveats or enhancements. |
| `PromptBuilderAgent`  | Takes the definition + review and crafts a smart search query or prompt for the final agent. |
| `ResearchAgent`       | Uses that prompt to search databases (like FRED) or simulate doing so, and writes a historical/contextual blog section. |
| *(Optional)* `WriterAgent` | Combines everything into a final blog post with engaging language and structure. |

---

### ✅ Why this is a great idea:
- **Modularity**: You can swap out any agent or improve its performance without rewriting the whole system.
- **Clarity**: You can trace output quality back to individual agents.
- **Collaboration**: Each agent is specialized, mirroring how human writers work.

---

### 🛠 Tools You Can Use

- **Hugging Face Pipelines** (`text2text-generation`) for each agent's task
- **Gradio or Streamlit** for interaction if needed
- **LangChain or Autogen** if you want orchestration (optional, can do manually too)
- **FAISS / Vector DB** for retrieval if you want memory or long-context injection
- **OpenAI function calling** (if you use GPT) or **Toolformer-style API tools** if going advanced

---

### 🔁 How to Structure the Code (Simplified Flow)

```python
# Step 1: Define the indicator
definition = definition_agent.run("Consumer Confidence Index")

# Step 2: Review the definition
review = review_agent.run(definition)

# Step 3: Build the prompt for a research query
search_prompt = prompt_builder_agent.run(definition, review)

# Step 4: Do research using that prompt (or simulate it with a local dataset)
blog_body = research_agent.run(search_prompt)

# Step 5: Compose the full blog post (optional)
final_post = writer_agent.run(definition, review, blog_body)
```

---

### 📚 Next Step

Would you like me to help you **start coding these agents step by step**, beginning with `DefinitionAgent`?

I can walk you through:
- Creating the pipeline
- Writing the prompt
- Running the model
- Structuring the input/output for reuse by the next agent


In [ ]:
# ✅ Critic Prompt Generator
def get_critic_prompt(definition_text):
    return f"""
You are a senior research analyst reviewing a blog draft for accuracy and clarity.

Please provide constructive feedback on the following definition of an economic indicator.
Assess whether it is clear, complete, and appropriate for a general audience. Suggest any improvements if needed.

Definition:
\"\"\"
{definition_text}
\"\"\"

Write your feedback in a friendly, professional tone.
""".strip()

# ✅ Run Critic Agent
def review_definition(definition_text):
    prompt = get_critic_prompt(definition_text)
    critic = pipeline("text2text-generation", model=MODEL_NAME)
    feedback = critic(prompt, max_new_tokens=300)[0]['generated_text'].strip()

    return {
        "definition_review": feedback,
        "original_definition": definition_text
    }

# ✅ Example Use
if __name__ == "__main__":
    indicator_name = "10-Year Treasury Yield"

    result = generate_definition(indicator_name)

    print("\n" + "="*40)
    print("✅ Writer Agent Output")
    print("="*40 + "\n")
    print(json.dumps(result, indent=2))

    review = review_definition(result["definition"])

    print("\n" + "="*40)
    print("✅ Critic Agent Output")
    print("="*40 + "\n")
    print(json.dumps(review, indent=2))



✅ Writer Agent Output

{
  "indicator": "10-Year Treasury Yield",
  "definition": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}

✅ Critic Agent Output

{
  "definition_review": "The 10-Year Treasury Yield is a crucial economic indicator that provides valuable insights into the health of the US economy. It is calculated by dividing the total amount of money in the Treasury by the number of years it h


========================================
✅ Writer Agent Output
========================================

{
  "indicator": "10-Year Treasury Yield",
  "definition": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}

========================================
✅ Critic Agent Output
========================================

{
  "definition_review": "The 10-Year Treasury Yield is a crucial economic indicator that provides valuable insights into the health of the US economy. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. This indicator is crucial for determining whether the economy is growing or shrinking, and it can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic growth. Overall, the definition of the 10-Year Treasury Yield",
  "original_definition": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}

You’re spot on — that’s a smart pivot. Here's why and what I'd recommend:

---

### 🧠 Observations

- **Agent 1 (DefinitionAgent)** is doing a solid job already: it gives you a blog-style intro to the indicator. ✅
- **Agent 2 (CriticAgent)** is just echoing the content, which is a known limitation of smaller instruction-tuned models like `LaMini-Flan-T5`. They’re not ideal for critical evaluation unless you fine-tune or go bigger.
  
---

### 🔄 New Plan: Repurpose Agent 2 as `ResearchAgent`

Let’s evolve Agent 2 into a **Research Agent** that takes the definition as context and enriches it with:
- **Recent data or trends from FRED**
- **Historical insight** (e.g., how the indicator behaved in past recessions)
- **Why it matters now** (economic relevance in the current cycle)

We can simulate this step with smart prompting first, and optionally add real search/RAG capabilities later.





In [ ]:
def get_research_prompt(indicator, definition_text):
    return f"""
You are a financial researcher preparing a blog post about the economic indicator: {indicator}.

Here is a basic definition to start with:
\"\"\"
{definition_text}
\"\"\"

Now expand on this with:
- Historical context and how the indicator has been used in the past
- Notable trends or movements in recent years (real or simulated)
- Why this indicator is important to watch in today's economic environment

Write in a blog-friendly tone suitable for general readers.
""".strip()


def generate_research_post(indicator, definition_text):
    prompt = get_research_prompt(indicator, definition_text)
    researcher = pipeline("text2text-generation", model=MODEL_NAME)
    response = researcher(prompt, max_new_tokens=500)[0]['generated_text'].strip()

    return {
        "indicator": indicator,
        "blog_section": response,
        "definition_used": definition_text
    }

if __name__ == "__main__":
    indicator_name = "10-Year Treasury Yield"
    result = generate_definition(indicator_name)

    print("\n" + "="*40)
    print("✅ Writer Agent Output")
    print("="*40 + "\n")
    print(json.dumps(result, indent=2))

    research = generate_research_post(indicator_name, result["definition"])

    print("\n" + "="*40)
    print("✅ Research Agent Output")
    print("="*40 + "\n")
    print(json.dumps(research, indent=2))



✅ Writer Agent Output

{
  "indicator": "10-Year Treasury Yield",
  "definition": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}

✅ Research Agent Output

{
  "indicator": "10-Year Treasury Yield",
  "blog_section": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by


========================================
✅ Writer Agent Output
========================================

{
  "indicator": "10-Year Treasury Yield",
  "definition": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}

========================================
✅ Research Agent Output
========================================

{
  "indicator": "10-Year Treasury Yield",
  "blog_section": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. This indicator is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic growth. Historical Context",
  "definition_used": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}

In [ ]:
!pip install -q python-dotenv fredapi yfinance pandas matplotlib

In [ ]:
from fredapi import Fred
import pandas as pd
import os
import requests
from dotenv import load_dotenv

# Path to the .env file (custom name)
env_file_path = '/content/FRED_API_KEY.env'

# Load the env file (point to your custom name)
load_dotenv("/content/FRED_API_KEY.env")

# Access the key
# Load your FRED API key from environment
FRED_API_KEY = os.getenv("FRED_API_KEY")
fred = Fred(api_key=FRED_API_KEY)
print("🔐 Loaded API key:", FRED_API_KEY[:6] + "..." if FRED_API_KEY else "Missing")

🔐 Loaded API key: 3f495a...


In [ ]:
def get_research_prompt_significance_only(indicator, definition_text):
    return f"""
You are writing a blog post for a general audience about the economic indicator: {indicator}.

You already have the following definition:
\"\"\"
{definition_text}
\"\"\"

Now expand on this definition by explaining:
- Why this indicator is important for understanding the economy
- What it typically signals to investors, analysts, and policymakers
- How it's commonly used in forecasting or decision-making
- Why this indicator is worth tracking today

Write in a friendly, educational blog-style tone for readers who are curious but not experts.
""".strip()

def generate_blog_section_significance(indicator, definition_text):
    prompt = get_research_prompt_significance_only(indicator, definition_text)
    researcher = pipeline("text2text-generation", model=MODEL_NAME)
    output = researcher(prompt, max_new_tokens=500)[0]['generated_text'].strip()

    return {
        "indicator": indicator,
        "blog_section": output,
        "definition_used": definition_text
    }


if __name__ == "__main__":
    indicator_name = "10-Year Treasury Yield"
    result = generate_definition(indicator_name)

    print("\n" + "="*40)
    print("✅ Writer Agent Output")
    print("="*40 + "\n")
    print(json.dumps(result, indent=2))

    blog = generate_blog_section_significance(indicator_name, result["definition"])

    print("\n" + "="*40)
    print("✅ Research Agent Output: Economic Significance")
    print("="*40 + "\n")
    print(blog["blog_section"])




✅ Writer Agent Output

{
  "indicator": "10-Year Treasury Yield",
  "definition": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}

✅ Research Agent Output: Economic Significance

The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the


========================================
✅ Writer Agent Output
========================================

{
  "indicator": "10-Year Treasury Yield",
  "definition": "The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. The 10-Year Treasury Yield is important because it provides insight into the overall health of the economy. It can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic"
}

========================================
✅ Research Agent Output: Economic Significance
========================================

The 10-Year Treasury Yield is a key economic indicator that measures the overall performance of the US Treasury. It is calculated by dividing the total amount of money in the Treasury by the number of years it has been in the Treasury. This indicator is important for understanding the economy because it can indicate whether the economy is growing or shrinking, and can also indicate whether the government is making progress towards achieving its economic goals. High Treasury Yields can be a sign of a healthy economy, as they can encourage borrowing and spending, which can stimulate economic growth. By tracking the 10-Year Treasury Yield,